In [1]:
import fitz #pymuPDF
import warnings
import re
import pandas as pd
import os
warnings.filterwarnings('ignore')
search_word = ["digital innovation",
"data infrastructure",
"business development",
"enabling environment",
"Data is defined",
"Definition of data"]

In [22]:
df_url = pd.read_csv('URL_added_v1.csv',encoding='ANSI')
df_url = df_url[df_url['URL'] != "#VALUE!"]
df_url = df_url[df_url['URL'].apply(lambda x: True if '.pdf' in x else False)]
df_url.reset_index(drop=True,inplace=True)

In [3]:
pdfs = set([int(i.split('.pdf')[0]) for i in os.listdir('pdf')[1:]])

error_pdf = set(df_url.index) - pdfs

In [4]:
df_processed = df_url.iloc[~df_url.index.isin(error_pdf)]

In [5]:
df_processed = df_processed[['Country/Territory', 'Title', 'Type', 'Link', 'URL']]

In [6]:
df_processed['PDF_Name'] = [str(i)+'.pdf' for i in list(df_processed.index)]

In [7]:
def search(filename,search_word):
    try:
        
        file = fitz.open(f'pdf/{filename}')
        list_file=[]
        for i in file.pages():
            list_file.append(i.getText())

        for j in range(file.pageCount):
            for i in ['\n','\xa0','\xad']:
                if i in list_file[j]:
                    list_file[j]=list_file[j].replace(i,' ')
        file.close()
        file_str = ''
        for i in list_file:
            file_str = file_str + i

        file_str = file_str.lower().strip()

        phrase_lower = search_word.lower()
        pattern1 = rf'\b{phrase_lower}\b'
        found1 = re.findall(pattern1, file_str)

        phrase_lower_s = phrase_lower+'s'
        pattern2 = rf'\b{phrase_lower_s}\b'
        found2 = re.findall(pattern2, file_str)

        if found2:
            new_phrase = '-'.join(phrase_lower_s.split())
            file_str = file_str.replace(phrase_lower_s,new_phrase)
            x = file_str.split()
            loc = x.index(new_phrase)
            text = ' '.join(x[loc+1: loc+51])
            return text

        elif found1:
            new_phrase = '-'.join(phrase_lower.split())
            file_str = file_str.replace(phrase_lower,new_phrase)
            x = file_str.split()
            loc = x.index(new_phrase)
            text = ' '.join(x[loc+1: loc+51])
            return text
        else:
            return float('NaN')
    except:
        return float('NaN')  

In [8]:
for word in search_word:
    df_processed[word] = df_processed['PDF_Name'].apply(lambda x: search(x,word))

mupdf: cmsOpenProfileFromMem failed
mupdf: cannot recognize xref format
mupdf: cmsOpenProfileFromMem failed
mupdf: cmsOpenProfileFromMem failed
mupdf: cannot recognize xref format
mupdf: cmsOpenProfileFromMem failed
mupdf: cmsOpenProfileFromMem failed
mupdf: cannot recognize xref format
mupdf: cmsOpenProfileFromMem failed
mupdf: cmsOpenProfileFromMem failed
mupdf: cannot recognize xref format
mupdf: cmsOpenProfileFromMem failed
mupdf: cmsOpenProfileFromMem failed
mupdf: cannot recognize xref format
mupdf: cmsOpenProfileFromMem failed
mupdf: cmsOpenProfileFromMem failed
mupdf: cannot recognize xref format
mupdf: cmsOpenProfileFromMem failed


In [10]:
df_processed.to_csv('processed.csv')

# Dont run the below code

In [18]:
assert 2 == 3

AssertionError: 

# pdf download code

In [12]:
from bs4 import BeautifulSoup
import urllib3
import requests

In [13]:
def pdf_download(url,filename):
    try:
        pdf_loc =  'pdf' + '/' + f'{filename}.pdf'
        s={'Accept': '*/*',
                            'Accept-Language': 'en-US,en;q=0.5',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:28.0) Gecko/20100101 Firefox/28.0',
                            'X-Requested-With': 'XMLHttpRequest'
                            }
        response = requests.get(url,headers=s)
        if response.status_code==200:
            with open(pdf_loc, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f'Error in {filename}: {url}')
            return False
    except:
        print(f'Error in {filename}: {url}')
        return False

In [ ]:
error = []
for indx in list(df_url.index):
    url = df_url.iloc[indx]['URL']
    flag = pdf_download(url,indx)
    if flag:
        print(f'file {indx}: {url} downloaded..')
    else:
        error.append(indx)

In [ ]:
len(error)

In [19]:
df_processed

,Country/Territory,Title,Type,Link,URL,PDF_Name,digital innovation,data infrastructure,business development,enabling environment,Data is defined,Definition of data
0,australia,Australia's Second National Plan of Action to ...,Part of Existing Sector Document (Agriculture/...,http://faolex.fao.org/docs/pdf/aus161858.pdf,http://faolex.fao.org/docs/pdf/aus161858.pdf,0.pdf,NaN,NaN,NaN,NaN,NaN,NaN
1,australia,Australia Tech Future,Part of National ICT/Digital Document,AUSTRALIA’S TECH FUTURE (industry.gov.au),https://www.industry.gov.au/sites/default/file...,1.pdf,NaN,NaN,NaN,australia’s tech future - 41 cyber security au...,NaN,NaN
2,australia,Digital Economy Strategy 2021,Part of National ICT/Digital Document,Digital Economy Strategy (pmc.gov.au),https://digitaleconomy.pmc.gov.au/sites/defaul...,2.pdf,NaN,NaN,NaN,NaN,NaN,NaN
3,australia,Digital Agriculture Strategy Victoria,Standalone,https://agriculture.vic.gov.au/__data/assets/p...,https://agriculture.vic.gov.au/__data/assets/p...,3.pdf,NaN,NaN,NaN,NaN,NaN,NaN
4,australia,Farm Data Code 2020 Edition 1,Standalone,Farm_Data_Code_Edition_1_WEB_FINAL.pdf (nff.or...,https://nff.org.au/wp-content/uploads/2020/02/...,4.pdf,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
205,vanuatu,Vanuatu National Information and Communication...,Part of National ICT/Digital Document,https://www.trbr.vu/attachments/category/166/v...,https://www.trbr.vu/attachments/category/166/v...,205.pdf,NaN,NaN,and employment opportunities) and democratic p...,"attractive to private investments, the governm...",NaN,NaN
206,vanuatu,National Fisheries Sector Policy 2016-2031,Part of Existing Sector Document (Agriculture/...,https://www.nab.vu/sites/default/files/documen...,https://www.nab.vu/sites/default/files/documen...,206.pdf,NaN,NaN,NaN,NaN,NaN,NaN
209,zambia,Second National Agricultural Policy,Part of Existing Sector Document (Agriculture/...,SECOND-NATIONAL-AGRICULTURAL-POLICY-2016.pdf (...,http://cbz.org.zm/public/downloads/SECOND-NATI...,209.pdf,NaN,NaN,NaN,NaN,NaN,NaN
211,botswana,National Information and Communication Technol...,Part of National ICT/Digital Document,https://publicadministration.un.org/unpsa/Port...,https://publicadministration.un.org/unpsa/Port...,211.pdf,NaN,NaN,NaN,for the growth of a sustainable ict industry. ...,NaN,NaN
